In [1]:
import pandas as pd
import numpy as np
#from fuzzywuzzy import process, fuzz
from rapidfuzz import process, fuzz
import re

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)

In [13]:
gtd_temp = pd.read_csv('gtd_baseAB.csv')
acled_temp = pd.read_csv('acled_baseAB.csv')
UCDP_temp = pd.read_csv('ucdp_baseAB.csv')

In [14]:
gtd_temp['geo_precision'].unique()

array([2, 1, 3, 4, 5])

In [16]:
acled_temp['geo_precision'].unique()

array([1, 2, 3, 4])

In [17]:
UCDP_temp['geo_precision'].unique()

array([2, 5, 3, 1, 4, 6, 7])

In [20]:
acled_temp[acled_temp['geo_precision'] == 4].shape

(52, 28)

In [10]:
gtd_temp[gtd_temp['longitude'].isnull() == True]['geo_precision']

,Unnamed: 0,eventid_fatality_count,eventid_source_no,eventid_original,grevd_id,year,month,day,midpoint_d,midpoint_m,midpoint_y,midpoint_date,latitude,longitude,country,width_range,time_precision,geo_precision,source1,source2,source3,source1_date,source2_date,source3_date,tags,Side_A,Side_B,date
429,4630,1,1002,201701040017,10022017010400170001,2017,1,4,NaN,NaN,NaN,NaN,NaN,NaN,Cameroon,0.0,NaN,5,NaN,NaN,NaN,2017-01-11,NaN,NaN,Unknown;;;Military;;;Bombing/Explosion;;;.,Unknown,NaN,2017-1-4
3604,38724,1,1002,201702180033,10022017021800330001,2017,2,18,NaN,NaN,NaN,NaN,NaN,NaN,Mali,0.0,NaN,5,NaN,NaN,NaN,2017-04-10,2017-04-05,NaN,Muslim extremists;;;Private Citizens & Property;;;Armed Assault;;;.,Muslim extremists,NaN,2017-2-18
3605,38725,2,1002,201702180033,10022017021800330002,2017,2,18,NaN,NaN,NaN,NaN,NaN,NaN,Mali,0.0,NaN,5,NaN,NaN,NaN,2017-04-10,2017-04-05,NaN,Muslim extremists;;;Private Citizens & Property;;;Armed Assault;;;.,Muslim extremists,NaN,2017-2-18
3606,38726,3,1002,201702180033,10022017021800330003,2017,2,18,NaN,NaN,NaN,NaN,NaN,NaN,Mali,0.0,NaN,5,NaN,NaN,NaN,2017-04-10,2017-04-05,NaN,Muslim extremists;;;Private Citizens & Property;;;Armed Assault;;;.,Muslim extremists,NaN,2017-2-18
3607,38727,4,1002,201702180033,10022017021800330004,2017,2,18,NaN,NaN,NaN,NaN,NaN,NaN,Mali,0.0,NaN,5,NaN,NaN,NaN,2017-04-10,2017-04-05,NaN,Muslim extremists;;;Private Citizens & Property;;;Armed Assault;;;.,Muslim extremists,NaN,2017-2-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49447,505891,1,1002,201812280031,10022018122800310001,2018,12,28,NaN,NaN,NaN,NaN,NaN,NaN,Iraq,0.0,NaN,5,NaN,NaN,NaN,2018-12-26,NaN,NaN,Kurdistan Workers' Party (PKK);;;Military;;;Unknown;;;.,Kurdistan Workers' Party (PKK),NaN,2018-12-28
49448,505892,2,1002,201812280031,10022018122800310002,2018,12,28,NaN,NaN,NaN,NaN,NaN,NaN,Iraq,0.0,NaN,5,NaN,NaN,NaN,2018-12-26,NaN,NaN,Kurdistan Workers' Party (PKK);;;Military;;;Unknown;;;.,Kurdistan Workers' Party (PKK),NaN,2018-12-28
49449,505893,3,1002,201812280031,10022018122800310003,2018,12,28,NaN,NaN,NaN,NaN,NaN,NaN,Iraq,0.0,NaN,5,NaN,NaN,NaN,2018-12-26,NaN,NaN,Kurdistan Workers' Party (PKK);;;Military;;;Unknown;;;.,Kurdistan Workers' Party (PKK),NaN,2018-12-28
49450,505894,4,1002,201812280031,10022018122800310004,2018,12,28,NaN,NaN,NaN,NaN,NaN,NaN,Iraq,0.0,NaN,5,NaN,NaN,NaN,2018-12-26,NaN,NaN,Kurdistan Workers' Party (PKK);;;Military;;;Unknown;;;.,Kurdistan Workers' Party (PKK),NaN,2018-12-28


In [3]:
# # Load GTD data
# gtd_df1 = pd.read_excel('globalterrorismdb_0221dist.xlsx', index_col=0)
# gtd_df2 = pd.read_excel('gtd1993_0221dist.xlsx', index_col=0)


In [4]:
# # Load UCDP data
# ucdp_df = pd.read_csv("ucdp-prio-acd-211.csv")
# ucdp_df.columns
# ucdp_df[["side_a","side_b"]]

In [3]:
# Load GreVD data
GReVD_stage_AB_df = pd.read_excel("grevd_stage_ab.xlsx")

In [4]:
# Find Side a and Side b
side_a = GReVD_stage_AB_df['tags'].apply(lambda x: x.split(";")[0])
side_b = GReVD_stage_AB_df['tags'].apply(lambda x: x.split(";")[1])

In [7]:
# # Potential source to play with for fuzzy matching
# GReVD_stage_AB_df['source1'][1000]
# GReVD_stage_AB_df['source2']
# GReVD_stage_AB_df['source3']

In [5]:
GReVD_naive_sides = pd.concat([side_a, side_b],keys=['tags_side_a', 'tags_side_b'], axis=1)

In [6]:
GReVD_stage_AB_df['Side_A'] = side_a

In [7]:
GReVD_stage_AB_df['Side_B'] = side_b

In [8]:
GReVD_stage_AB_df[['year','month','day']]

,year,month,day
0,2017,1,1
1,2017,1,1
2,2017,1,1
3,2017,1,1
4,2017,1,1
...,...,...,...
654348,2020,1,1
654349,2020,1,1
654350,2020,1,1
654351,2020,1,1


In [10]:
GReVD_stage_AB_df['date'] = GReVD_stage_AB_df[['year','month','day']].apply(
    lambda x: '-'.join(x.dropna().astype(str)),
    axis=1
)

In [17]:
acled_df = GReVD_stage_AB_df[GReVD_stage_AB_df['eventid_source_no'] == 1001]
gtd_df = GReVD_stage_AB_df[GReVD_stage_AB_df['eventid_source_no'] == 1002]
ucdp_df = GReVD_stage_AB_df[GReVD_stage_AB_df['eventid_source_no'] == 1003]

In [ ]:
acled_df.to_csv('acled_baseAB.csv')
gtd_df.to_csv('gtd_baseAB.csv')
ucdp_df.to_csv('ucdp_baseAB.csv')
GReVD_stage_AB_df.to_csv('grevd_baseAB.csv')

In [8]:
GReVD_naive_sides

,tags_side_a,tags_side_b
0,Unidentified Armed Group (Nigeria),Civilians (Nigeria)
1,Unidentified Armed Group (Nigeria),Civilians (Nigeria)
2,Unidentified Armed Group (Nigeria),Civilians (Nigeria)
3,Military Forces of Iraq (2014-2018),Islamic State (Iraq)
4,Military Forces of Iraq (2014-2018),Islamic State (Iraq)
...,...,...
654348,Unidentified Armed Group (Colombia),Civilians (Colombia)
654349,Unidentified Armed Group (Mexico),Civilians (Mexico)
654350,AQAP: Al Qaeda in the Arabian Peninsula,Islamic State (Yemen)
654351,JNIM: Group for Support of Islam and Muslims,Islamic State (Greater Sahara)


In [9]:
# Function cell to get the final sides from local GReVD data
def get_final_sides(column_name_a, column_name_b, df):
    temp_unique_list = []
    for i in df[column_name_a]:
        if i not in temp_unique_list:
            temp_unique_list.append(i)

    temp_unique_list_b = []
    for i in df[column_name_b]:
        if i not in temp_unique_list_b:
            temp_unique_list_b.append(i)

    final_lst = []
    for i in temp_unique_list:
        if i not in final_lst:
            final_lst.append(i)

    for i in temp_unique_list_b:
        if i not in final_lst:
            final_lst.append(i)

    print('Length of final list: ', len(final_lst))  
    print('Ratio of unique actors', len(final_lst)/len(df))

    # Sort the final list by alphabetical order
    final_lst.sort()      
    return final_lst

In [10]:
final_list = get_final_sides('tags_side_a', 'tags_side_b', GReVD_naive_sides)

Length of final list:  6104
Ratio of unique actors 0.009328298334385263


In [11]:
# Load the actor_taxonomy.csv file and create a dataframe
actor_taxonomy = pd.read_csv('actor_taxonomy.csv')

In [12]:
# Find the columns of actors in the actor_taxonomy.csv file
actor_taxonomy['base.categories']

0                    "Al Shabaab" Militia
1                     "Janjaweed" Militia
2                       "Taliban" Militia
3                 101 Battalion (Tajoura)
4       Aandonga Ethnic Militia (Namibia)
                      ...                
4076                      Zintani Militia
4077          Zionist Resistance Fighters
4078                       Zulu Militants
4079                          Zulu Miners
4080                           Zwai Tribe
Name: base.categories, Length: 4081, dtype: object

In [13]:
# Naive way of finding actors that are not in the actor_taxonomy.csv file

temp_not_in_texonomy_lst = []
for i in final_list:
    if i not in actor_taxonomy['base.categories'].to_list():
        temp_not_in_texonomy_lst.append(i)
print('Size of the actors that are not in taxonomy list: ', len(temp_not_in_texonomy_lst))

Size of the actors that are not in taxonomy list:  5334


In [14]:
# A function cell for creating match comparison dataframe with fuzzy matching
def get_fuzzy_match_df(lst,lower_score_threshold,upper_score_threshold):
    score_sort = [(x,) + i
             for x in lst 
             for i in process.extract(x, lst, scorer=fuzz.token_set_ratio)] # change from token sort to token set
    #Create a dataframe from the tuples
    similarity_sort = pd.DataFrame(score_sort)
    similarity_sort.columns = ['side_sort','match_sort','score_sort','usesless/meaningless']
    similarity_sort['sorted_side_sort'] = np.minimum(similarity_sort['side_sort'], similarity_sort['match_sort'])

    # Find the match within the final list with a low threshold of 80% match score
    high_score_sort = similarity_sort[
        #(similarity_sort['score_sort'] >= 80) &
                    (similarity_sort['side_sort'] !=  similarity_sort['match_sort']) &
                    (similarity_sort['sorted_side_sort'] != similarity_sort['match_sort'])]
    high_score_sort = high_score_sort.drop('sorted_side_sort',axis=1).copy()

    # write a function with similar socre threshold as parameters and thus display the entire column
    def get_similar_sides():
        temp_lower_bound_set = high_score_sort[high_score_sort['score_sort'] >= lower_score_threshold]
        temp_upper_bound_set = temp_lower_bound_set[temp_lower_bound_set['score_sort'] < upper_score_threshold]
        result = temp_upper_bound_set.groupby(['side_sort','score_sort']).agg(
                            {'match_sort': ', '.join}).sort_values(
                            ['score_sort'], ascending=False)
        return result
    return get_similar_sides()


In [15]:
# Combine both the taxonomy and the original unique actor list and run the same process as above to get the final list
joined_final_list_local_and_taxonomy =  []
for i in final_list:
    if i not in joined_final_list_local_and_taxonomy:
        joined_final_list_local_and_taxonomy.append(i)
for i in actor_taxonomy['base.categories']:
    if i not in joined_final_list_local_and_taxonomy:
        joined_final_list_local_and_taxonomy.append(i)

# Pop the index of 6104 becasue it is weird (Contains all of the taxonomy actors)
# joined_final_list_local_and_taxonomy[6105]


In [16]:
# Create a dataframe with sorted actors in the final_list
matched_df_75_100 = get_fuzzy_match_df(joined_final_list_local_and_taxonomy,75,100)

In [17]:
for i in matched_df_75_100['match_sort'].to_list():
    if i in actor_taxonomy:
        print(actor_taxonomy.iloc(i)['Level_1'])

In [18]:
matched_df_75_100_with_sort = get_fuzzy_match_df(joined_final_list_local_and_taxonomy,75,100)

In [19]:
matched_df_75_100_with_sort.head()

,,match_sort
side_sort,score_sort,
SPLM-N-NMLC: Sudan People's Liberation Movement (North) (Nuba Mountains Liberation Council Faction),99.393939,SPLM-N-NMLC: Sudan Peoples Liberation Movement (North) (Nuba Mountains Liberation Council Faction)
DMLEK: Democratic Movement for the Liberation of Eritrean Kunama,99.212598,Democratic Movement for the Liberation of the Eritrean Kunamas (DMLEK)
SPLM-N: Sudan People's Liberation Movement (North),98.924731,SPLM-N: Sudan Peoples Liberation Movement (North)
Military Forces of Algeria (1999-) Border Guard,98.876404,Military Forces of Algeria (1999-) Border Guards
PUDEMO: Peoples United Democratic Movement,98.795181,People's United Democratic Movement (PUDEMO)


In [20]:
# For above the threshold, identify similar ones, and for below the threshold(probably all unique) and append into the final list and send it to Gary
# from index 409 and above are eyeballed as the same, anything below should be unique, score as 95.3488372093
matched_df_75_100_with_sort['match_sort'] 

side_sort                                                                                             score_sort
 SPLM-N-NMLC: Sudan People's Liberation Movement (North) (Nuba Mountains Liberation Council Faction)  99.393939     SPLM-N-NMLC: Sudan Peoples Liberation Movement (North) (Nuba Mountains Liberation Council Faction)
DMLEK: Democratic Movement for the Liberation of Eritrean Kunama                                      99.212598                                 Democratic Movement for the Liberation of the Eritrean Kunamas (DMLEK)
 SPLM-N: Sudan People's Liberation Movement (North)                                                   98.924731                                                      SPLM-N: Sudan Peoples Liberation Movement (North)
Military Forces of Algeria (1999-) Border Guard                                                       98.876404                                                       Military Forces of Algeria (1999-) Border Guards
PUDEMO: Peoples United Demo

In [21]:
def get_fuzzy_match_df_final(lst,lower_score_threshold):
    score_sort = [(x,) + i
             for x in lst 
             for i in process.extract(x, lst, scorer=fuzz.token_set_ratio)] # change from token sort to token set
    #Create a dataframe from the tuples
    similarity_sort = pd.DataFrame(score_sort)
    similarity_sort.columns = ['side_sort','match_sort','score_sort','usesless/meaningless']
    similarity_sort['sorted_side_sort'] = np.minimum(similarity_sort['side_sort'], similarity_sort['match_sort'])

    # Find the match within the final list with a low threshold of 80% match score
    high_score_sort = similarity_sort[
        (similarity_sort['score_sort'] >= lower_score_threshold) &
                    (similarity_sort['side_sort'] !=  similarity_sort['match_sort']) &
                    (similarity_sort['sorted_side_sort'] != similarity_sort['match_sort'])]
    high_score_sort = high_score_sort.drop('sorted_side_sort',axis=1).copy()

    return high_score_sort

In [22]:
temp_same_above_95 = get_fuzzy_match_df_final(joined_final_list_local_and_taxonomy,95.3488372093)

In [23]:
temp_match_count_list = []
for i in temp_same_above_95['match_sort']:
    if i in actor_taxonomy['base.categories'].to_list():
        temp_match_count_list.append(1)
    else:
        temp_match_count_list.append(0)

temp_side_count_list = []
for i in temp_same_above_95['side_sort']:
    if i in actor_taxonomy['base.categories'].to_list():
        temp_side_count_list.append(1)
    else:
        temp_side_count_list.append(0)

temp_same_above_95['match_sort_in_taxonomy'] = temp_match_count_list
temp_same_above_95['side_sort_in_taxonomy'] = temp_side_count_list



In [24]:
temp_same_above_95

,side_sort,match_sort,score_sort,usesless/meaningless,match_sort_in_taxonomy,side_sort_in_taxonomy
31,3rd Corps,3rd Corps,100.0,2674,0,0
46,6th Infantry Brigade,6th Infantry Brigade,100.0,2676,0,0
51,7th Brigade,7th Brigade,100.0,2677,0,0
61,AA,ULA/AA: United League of Arakan/Arakan Army,100.0,2461,0,0
62,AA,Arakan Army (AA),100.0,2920,0,0
...,...,...,...,...,...,...
46460,Tutsi,Tutsi Ethnic Militia (Democratic Republic of Congo),100.0,8318,1,1
46461,Tutsi,Tutsi Ethnic Militia (Rwanda),100.0,8319,1,1
46621,War Veterans,ZNLWVA: Zimbabwe National Liberation War Veterans Association,100.0,8516,1,1
46630,West Side Boys,West Side Boys Militia,100.0,8479,1,1


In [25]:
index_of_matched = np.where(np.array(temp_same_above_95['match_sort_in_taxonomy'].to_list()) == 1)[0]
temp_same_above_95.iloc[index_of_matched]['match_sort']

76                                                         ABC: All Basotho Convention
94                                                                                 ADF
98                                                                                 ADF
99       ADF-NALU: Allied Democratic Forces-National Army for the Liberation of Uganda
103                                                                                ADF
                                             ...                                      
46460                              Tutsi Ethnic Militia (Democratic Republic of Congo)
46461                                                    Tutsi Ethnic Militia (Rwanda)
46621                    ZNLWVA: Zimbabwe National Liberation War Veterans Association
46630                                                           West Side Boys Militia
46631                                             West Side Boys Militia (Demobilized)
Name: match_sort, Length: 2812, dtype: obje

In [26]:
matched_taxonomy = actor_taxonomy[actor_taxonomy["base.categories"].isin(temp_same_above_95.iloc[index_of_matched]['match_sort'].to_list())]

In [27]:
temp_same_above_95.merge(matched_taxonomy, left_on='match_sort', right_on='base.categories', how='left')

,side_sort,match_sort,score_sort,usesless/meaningless,match_sort_in_taxonomy,side_sort_in_taxonomy,data.source,base.categories,Level_1,Level_2,Level_3
0,3rd Corps,3rd Corps,100.0,2674,0,0,NaN,NaN,NaN,NaN,NaN
1,6th Infantry Brigade,6th Infantry Brigade,100.0,2676,0,0,NaN,NaN,NaN,NaN,NaN
2,7th Brigade,7th Brigade,100.0,2677,0,0,NaN,NaN,NaN,NaN,NaN
3,AA,ULA/AA: United League of Arakan/Arakan Army,100.0,2461,0,0,NaN,NaN,NaN,NaN,NaN
4,AA,Arakan Army (AA),100.0,2920,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5686,Tutsi,Tutsi Ethnic Militia (Democratic Republic of Congo),100.0,8318,1,1,ACLED,Tutsi Ethnic Militia (Democratic Republic of Congo),armed group,violent political groups,violent groups
5687,Tutsi,Tutsi Ethnic Militia (Rwanda),100.0,8319,1,1,ACLED,Tutsi Ethnic Militia (Rwanda),armed group,violent political groups,violent groups
5688,War Veterans,ZNLWVA: Zimbabwe National Liberation War Veterans Association,100.0,8516,1,1,ACLED,ZNLWVA: Zimbabwe National Liberation War Veterans Association,civilians,nonviolent civilian groups,nonviolent groups
5689,West Side Boys,West Side Boys Militia,100.0,8479,1,1,ACLED,West Side Boys Militia,armed group,violent political groups,violent groups


In [28]:
print('side matches: ', sum(temp_side_count_list))
print('match matches: ', sum(temp_match_count_list))

side matches:  1423
match matches:  2812


In [29]:
from itertools import starmap
from operator import or_
import xlsxwriter

In [30]:
# checking if there are dupllicates in the list
temp_combined = list(starmap(or_, zip(temp_side_count_list, temp_match_count_list)))

In [31]:
temp_same_above_95['side_sort'].iloc[1]

' 6th Infantry Brigade'

In [32]:
def indices(lst, item):
   return [i for i, x in enumerate(lst) if x == item]
 
indeices_list = indices(temp_combined, 0)

In [33]:
# Not in actor_taxonomy but are duplicated, so should be added to the final actor list
temp_actor_final_list_for_Gary = []
for i in indeices_list:
    temp_actor_final_list_for_Gary.append(temp_same_above_95['side_sort'].iloc[i])

In [34]:
len(temp_actor_final_list_for_Gary)

2611

In [35]:
len(list(set(temp_actor_final_list_for_Gary)))

1702

In [36]:
def get_fuzzy_match_df_final_lowerones(lst,upper_score_threshold):
    score_sort = [(x,) + i
             for x in lst 
             for i in process.extract(x, lst, scorer=fuzz.token_set_ratio)] # change from token sort to token set
    #Create a dataframe from the tuples
    similarity_sort = pd.DataFrame(score_sort)
    similarity_sort.columns = ['side_sort','match_sort','score_sort','usesless/meaningless']
    similarity_sort['sorted_side_sort'] = np.minimum(similarity_sort['side_sort'], similarity_sort['match_sort'])

    # Find the match within the final list with a low threshold of 80% match score
    high_score_sort = similarity_sort[
        (similarity_sort['score_sort'] <= upper_score_threshold) &
                    (similarity_sort['side_sort'] !=  similarity_sort['match_sort']) &
                    (similarity_sort['sorted_side_sort'] != similarity_sort['match_sort'])]
    high_score_sort = high_score_sort.drop('sorted_side_sort',axis=1).copy()

    return high_score_sort

In [37]:
temp_same_below_95 = get_fuzzy_match_df_final_lowerones(joined_final_list_local_and_taxonomy,95.3488372093)

In [38]:
temp_same_below_95.merge(actor_taxonomy, left_on='match_sort', right_on='base.categories', how='left')

,side_sort,match_sort,score_sort,usesless/meaningless,data.source,base.categories,Level_1,Level_2,Level_3
0,,,0.000000,1,NaN,NaN,NaN,NaN,NaN
1,,1st Coastal Division,0.000000,2,NaN,NaN,NaN,NaN,NaN
2,,1st Infantry Division,0.000000,3,NaN,NaN,NaN,NaN,NaN
3,,1st Regiment,0.000000,4,NaN,NaN,NaN,NaN,NaN
4,,1st Coastal Division,0.000000,2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10270,United Democratic Front (UDF),United People's Democratic Front (UPDF) - Bangladesh,92.000000,6020,NaN,NaN,NaN,NaN,NaN
10271,Urhobo Gbagbako,Urhobo Revolutionary Army,57.142857,9321,GTD,Urhobo Revolutionary Army,armed group,violent political groups,violent groups
10272,West Nile Bank Front (WNBF),Western Somalia Liberation Front,59.649123,9327,GTD,Western Somalia Liberation Front,armed group,violent political groups,violent groups
10273,Zimbabwe African Nationalist Union (ZANU),Zimbabwe African People's Union,83.018868,9341,GTD,Zimbabwe African People's Union,armed group,violent political groups,violent groups


In [39]:
temp_same_below_95.merge(actor_taxonomy, left_on='match_sort', right_on='base.categories', how='left')['Level_1'].isnull().sum()

5285

In [40]:
temp_same_below_95.merge(actor_taxonomy, left_on='match_sort', right_on='base.categories', how='left')\
    [temp_same_below_95.merge(actor_taxonomy, left_on='match_sort', right_on='base.categories', how='left')['Level_1'].isnull() == False]

,side_sort,match_sort,score_sort,usesless/meaningless,data.source,base.categories,Level_1,Level_2,Level_3
16,1st Regiment,Tribesmen,57.142857,5784,GTD,Tribesmen,armed group,violent political groups,violent groups
17,1st Regiment,Sabaot tribesmen,57.142857,9224,GTD,Sabaot tribesmen,armed group,violent political groups,violent groups
25,4th Legion,Islamic Legion,75.000000,9036,GTD,Islamic Legion,armed group,violent political groups,violent groups
36,9th Division Special Forces,Military Forces of Democratic Republic of Congo (1997-2001) Special Presidential Division,92.000000,7333,ACLED,Military Forces of Democratic Republic of Congo (1997-2001) Special Presidential Division,military,violent political groups,violent groups
39,AA,MAA,80.000000,8727,GED,MAA,armed group,violent political groups,violent groups
...,...,...,...,...,...,...,...,...,...
10267,Ugandan Rebels,Yau Yau Rebels,75.000000,8496,ACLED,Yau Yau Rebels,armed group,violent political groups,violent groups
10271,Urhobo Gbagbako,Urhobo Revolutionary Army,57.142857,9321,GTD,Urhobo Revolutionary Army,armed group,violent political groups,violent groups
10272,West Nile Bank Front (WNBF),Western Somalia Liberation Front,59.649123,9327,GTD,Western Somalia Liberation Front,armed group,violent political groups,violent groups
10273,Zimbabwe African Nationalist Union (ZANU),Zimbabwe African People's Union,83.018868,9341,GTD,Zimbabwe African People's Union,armed group,violent political groups,violent groups


In [47]:
final_actor_df_for_Gary_May_2.merge(filled_lvel_basedon_match_sort, left_on='missing_actors', right_on='side_sort', how='left')

,missing_actors,side_sort,match_sort,score_sort,usesless/meaningless,data.source,base.categories,Level_1,Level_2,Level_3
0,Lela Chanco Communal Militia (Guatemala),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LeJ: Lashkar-e-Jabbar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Police Forces of Myanmar (2016-) Border Guard Police,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,UPLC,UPLC,UPC,85.714286,8837.0,GED,UPC,political party,nonviolent political groups,nonviolent groups
4,Police Forces of Ethiopia (2018-) Liyu Police,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3255,Police Forces of Uzbekistan (2016-) Prison Guards,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3256,Rohingya extremists,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3257,Qeerroo Youth Militia,Qeerroo Youth Militia,Koras Youth Militia,81.250000,7093.0,ACLED,Koras Youth Militia,armed group,violent political groups,violent groups
3258,Indigenous Militia (Mexico),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
filled_lvel_basedon_match_sort = temp_same_below_95.merge(actor_taxonomy, left_on='match_sort', right_on='base.categories', how='left')\
    [temp_same_below_95.merge(actor_taxonomy, left_on='match_sort', right_on='base.categories', how='left')['Level_1']\
        .isnull() == False].drop_duplicates(subset=['side_sort'])
filled_lvel_basedon_match_sort

,side_sort,match_sort,score_sort,usesless/meaningless,data.source,base.categories,Level_1,Level_2,Level_3
16,1st Regiment,Tribesmen,57.142857,5784,GTD,Tribesmen,armed group,violent political groups,violent groups
25,4th Legion,Islamic Legion,75.000000,9036,GTD,Islamic Legion,armed group,violent political groups,violent groups
36,9th Division Special Forces,Military Forces of Democratic Republic of Congo (1997-2001) Special Presidential Division,92.000000,7333,ACLED,Military Forces of Democratic Republic of Congo (1997-2001) Special Presidential Division,military,violent political groups,violent groups
39,AA,MAA,80.000000,8727,GED,MAA,armed group,violent political groups,violent groups
42,AAK: Ajnad al Kavkaz,Ajnad Misr,66.666667,6177,ACLED,Ajnad Misr,armed group,violent political groups,violent groups
...,...,...,...,...,...,...,...,...,...
10267,Ugandan Rebels,Yau Yau Rebels,75.000000,8496,ACLED,Yau Yau Rebels,armed group,violent political groups,violent groups
10271,Urhobo Gbagbako,Urhobo Revolutionary Army,57.142857,9321,GTD,Urhobo Revolutionary Army,armed group,violent political groups,violent groups
10272,West Nile Bank Front (WNBF),Western Somalia Liberation Front,59.649123,9327,GTD,Western Somalia Liberation Front,armed group,violent political groups,violent groups
10273,Zimbabwe African Nationalist Union (ZANU),Zimbabwe African People's Union,83.018868,9341,GTD,Zimbabwe African People's Union,armed group,violent political groups,violent groups


In [67]:
temp_same_below_95.merge(actor_taxonomy, left_on='match_sort', right_on='base.categories', how='left')\
    [temp_same_below_95.merge(actor_taxonomy, left_on='match_sort', right_on='base.categories', how='left')['Level_1'].isnull() == False]

,side_sort,match_sort,score_sort,usesless/meaningless,data.source,base.categories,Level_1,Level_2,Level_3
16,1st Regiment,Tribesmen,57.142857,5784,GTD,Tribesmen,armed group,violent political groups,violent groups
17,1st Regiment,Sabaot tribesmen,57.142857,9224,GTD,Sabaot tribesmen,armed group,violent political groups,violent groups
25,4th Legion,Islamic Legion,75.000000,9036,GTD,Islamic Legion,armed group,violent political groups,violent groups
36,9th Division Special Forces,Military Forces of Democratic Republic of Congo (1997-2001) Special Presidential Division,92.000000,7333,ACLED,Military Forces of Democratic Republic of Congo (1997-2001) Special Presidential Division,military,violent political groups,violent groups
39,AA,MAA,80.000000,8727,GED,MAA,armed group,violent political groups,violent groups
...,...,...,...,...,...,...,...,...,...
10267,Ugandan Rebels,Yau Yau Rebels,75.000000,8496,ACLED,Yau Yau Rebels,armed group,violent political groups,violent groups
10271,Urhobo Gbagbako,Urhobo Revolutionary Army,57.142857,9321,GTD,Urhobo Revolutionary Army,armed group,violent political groups,violent groups
10272,West Nile Bank Front (WNBF),Western Somalia Liberation Front,59.649123,9327,GTD,Western Somalia Liberation Front,armed group,violent political groups,violent groups
10273,Zimbabwe African Nationalist Union (ZANU),Zimbabwe African People's Union,83.018868,9341,GTD,Zimbabwe African People's Union,armed group,violent political groups,violent groups


In [66]:
filled_lvel_basedon_side_sort = temp_same_below_95.merge(actor_taxonomy, left_on='side_sort', right_on='base.categories', how='left')\
    [temp_same_below_95.merge(actor_taxonomy, left_on='side_sort', right_on='base.categories', how='left')['Level_1']\
        .isnull() == False].drop_duplicates(subset=['match_sort'])
filled_lvel_basedon_side_sort

,side_sort,match_sort,score_sort,usesless/meaningless,data.source,base.categories,Level_1,Level_2,Level_3
4344,AGUDM: Patriotic Ginbot 7 for Unity and Democratic Movement,Patriotic Arab Democratic Movement,92.063492,4970,ACLED,AGUDM: Patriotic Ginbot 7 for Unity and Democratic Movement,armed group,violent political groups,violent groups
4345,AGUDM: Patriotic Ginbot 7 for Unity and Democratic Movement,Patriotic Ginbot 7 Movement for Unity and Democracy (PGMUD),86.956522,4971,ACLED,AGUDM: Patriotic Ginbot 7 for Unity and Democratic Movement,armed group,violent political groups,violent groups
4346,AGUDM: Patriotic Ginbot 7 for Unity and Democratic Movement,Somali Patriotic Movement,83.720930,9239,ACLED,AGUDM: Patriotic Ginbot 7 for Unity and Democratic Movement,armed group,violent political groups,violent groups
4354,ANCYL: African National Congress Youth League,African National Congress (South Africa),82.926829,8852,ACLED,ANCYL: African National Congress Youth League,civilians,nonviolent civilian groups,nonviolent groups
4355,ANCYL: African National Congress Youth League,NCP: National Congress Party,77.272727,4740,ACLED,ANCYL: African National Congress Youth League,civilians,nonviolent civilian groups,nonviolent groups
...,...,...,...,...,...,...,...,...,...
10221,Tripoli Revolutionaries Battalion,Tripoli Revolutionaries Brigade,85.185185,5787,GTD,Tripoli Revolutionaries Battalion,armed group,violent political groups,violent groups
10222,Tripoli Revolutionaries Battalion,Tripoli Revolutionaries Brigade (First Division),82.142857,8308,GTD,Tripoli Revolutionaries Battalion,armed group,violent political groups,violent groups
10230,Unidentified Infiltrators,Unidentified Tribal Militia (Iran),73.684211,6008,GTD,Unidentified Infiltrators,unknown,unknown,violent groups
10231,Unidentified Infiltrators,Unidentified Military Forces,71.698113,6006,GTD,Unidentified Infiltrators,unknown,unknown,violent groups


In [92]:
semi_final_bin = filled_lvel_basedon_match_sort.merge(filled_lvel_basedon_side_sort, left_on='match_sort', right_on='side_sort', how='left')\
    [filled_lvel_basedon_match_sort.merge(filled_lvel_basedon_side_sort, left_on='match_sort', right_on='side_sort', how='left')['Level_1_x']\
        .isnull() == False][["side_sort_x",	"match_sort_x","Level_1_x","Level_2_x","Level_3_x"]]
semi_final_bin

,side_sort_x,match_sort_x,Level_1_x,Level_2_x,Level_3_x
0,1st Regiment,Tribesmen,armed group,violent political groups,violent groups
1,4th Legion,Islamic Legion,armed group,violent political groups,violent groups
2,4th Legion,Islamic Legion,armed group,violent political groups,violent groups
3,9th Division Special Forces,Military Forces of Democratic Republic of Congo (1997-2001) Special Presidential Division,military,violent political groups,violent groups
4,AA,MAA,armed group,violent political groups,violent groups
...,...,...,...,...,...
3411,Ugandan Rebels,Yau Yau Rebels,armed group,violent political groups,violent groups
3412,Urhobo Gbagbako,Urhobo Revolutionary Army,armed group,violent political groups,violent groups
3413,West Nile Bank Front (WNBF),Western Somalia Liberation Front,armed group,violent political groups,violent groups
3414,Zimbabwe African Nationalist Union (ZANU),Zimbabwe African People's Union,armed group,violent political groups,violent groups


In [89]:
counter = 0
for i in semi_final_bin['match_sort_x'].to_list():
    if i in final_actor_df_for_Gary_May_2['missing_actors'].to_list():
        counter += 1
print(counter)

0


In [94]:
final_actor_df_for_Gary_May_2

,missing_actors
0,Lela Chanco Communal Militia (Guatemala)
1,LeJ: Lashkar-e-Jabbar
2,Police Forces of Myanmar (2016-) Border Guard Police
3,UPLC
4,Police Forces of Ethiopia (2018-) Liyu Police
...,...
3255,Police Forces of Uzbekistan (2016-) Prison Guards
3256,Rohingya extremists
3257,Qeerroo Youth Militia
3258,Indigenous Militia (Mexico)


In [93]:
final_actor_df_for_Gary_May_2.merge(semi_final_bin, left_on='missing_actors', right_on='side_sort_x', how='left')

,missing_actors,side_sort_x,match_sort_x,Level_1_x,Level_2_x,Level_3_x
0,Lela Chanco Communal Militia (Guatemala),NaN,NaN,NaN,NaN,NaN
1,LeJ: Lashkar-e-Jabbar,NaN,NaN,NaN,NaN,NaN
2,Police Forces of Myanmar (2016-) Border Guard Police,NaN,NaN,NaN,NaN,NaN
3,UPLC,UPLC,UPC,political party,nonviolent political groups,nonviolent groups
4,Police Forces of Ethiopia (2018-) Liyu Police,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
3332,Police Forces of Uzbekistan (2016-) Prison Guards,NaN,NaN,NaN,NaN,NaN
3333,Rohingya extremists,NaN,NaN,NaN,NaN,NaN
3334,Qeerroo Youth Militia,Qeerroo Youth Militia,Koras Youth Militia,armed group,violent political groups,violent groups
3335,Indigenous Militia (Mexico),NaN,NaN,NaN,NaN,NaN


In [76]:
final_actor_df_for_Gary_May_2.merge(semi_final_bin, left_on='missing_actors', right_on='match_sort_x', how='left')

,missing_actors,side_sort_x,match_sort_x,score_sort_x,usesless/meaningless_x,data.source_x,base.categories_x,Level_1_x,Level_2_x,Level_3_x,side_sort_y,match_sort_y,score_sort_y,usesless/meaningless_y,data.source_y,base.categories_y,Level_1_y,Level_2_y,Level_3_y
0,Lela Chanco Communal Militia (Guatemala),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LeJ: Lashkar-e-Jabbar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Police Forces of Myanmar (2016-) Border Guard Police,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,UPLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Police Forces of Ethiopia (2018-) Liyu Police,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,Police Forces of Uzbekistan (2016-) Prison Guards,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3256,Rohingya extremists,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3257,Qeerroo Youth Militia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3258,Indigenous Militia (Mexico),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
concatted_filled_df = pd.concat([filled_lvel_basedon_match_sort,filled_lvel_basedon_side_sort],keys=['side_sort', 'match_sort'])
concatted_filled_df

side_sort  \
side_sort  16                                  1st Regiment   
           25                                    4th Legion   
           36                   9th Division Special Forces   
           39                                            AA   
           42                          AAK: Ajnad al Kavkaz   
...                                                     ...   
match_sort 10221          Tripoli Revolutionaries Battalion   
           10222          Tripoli Revolutionaries Battalion   
           10230                  Unidentified Infiltrators   
           10231                  Unidentified Infiltrators   
           10235  Zimbabwe African Nationalist Union (ZANU)   

                                                                                                 match_sort  \
side_sort  16                                                                                     Tribesmen   
           25                                                                                Islamic Legion   
           36     Military Forces of Democratic Republic of Congo (1997-2001) Special Presidential Division   
           39                                                                                           MAA   
           42                                                                                    Ajnad Misr   
...                                                                                                     ...   
match_sort 10221                                                            Tripoli Revolutionaries Brigade   
           10222                                           Tripoli Revolutionaries Brigade (First Division)   
           10230                                                         Unidentified Tribal Militia (Iran)   
           10231                                                               Unidentified Military Forces   
           10235                                                            Zimbabwe African People's Union   

                  score_sort  usesless/meaningless data.source  \
side_sort  16      57.142857                  5784         GTD   
           25      75.000000                  9036         GTD   
           36      92.000000                  7333       ACLED   
           39      80.000000                  8727         GED   
           42      66.666667                  6177       ACLED   
...                      ...                   ...         ...   
match_sort 10221   85.185185                  5787         GTD   
           10222   82.142857                  8308         GTD   
           10230   73.684211                  6008         GTD   
           10231   71.698113                  6006         GTD   
           10235   83.018868                  9341         GTD   

                                                                                            base.categories  \
side_sort  16                                                                                     Tribesmen   
           25                                                                                Islamic Legion   
           36     Military Forces of Democratic Republic of Congo (1997-2001) Special Presidential Division   
           39                                                                                           MAA   
           42                                                                                    Ajnad Misr   
...                                                                                                     ...   
match_sort 10221                                                          Tripoli Revolutionaries Battalion   
           10222                                                          Tripoli Revolutionaries Battalion   
           10230                                                                  Unidentified Infiltrators   
           10231                                         

In [52]:
merged_filled_side = final_actor_df_for_Gary_May_2.merge(concatted_filled_df, left_on='missing_actors', right_on='side_sort', how='left')

In [53]:
merged_filled_match = final_actor_df_for_Gary_May_2.merge(concatted_filled_df, left_on='missing_actors', right_on='match_sort', how='left')

In [98]:
#pd.concat([merged_filled_side,merged_filled_match])
trial_bin = pd.concat([merged_filled_side,merged_filled_match])[pd.concat([merged_filled_side,merged_filled_match])['Level_1'].notna()]
trial_bin

,missing_actors,side_sort,match_sort,score_sort,usesless/meaningless,data.source,base.categories,Level_1,Level_2,Level_3
3,UPLC,UPLC,UPC,85.714286,8837.0,GED,UPC,political party,nonviolent political groups,nonviolent groups
13,RCSS/SSA-S: Restoration Council of Shan State/Shan State Army-South,RCSS/SSA-S: Restoration Council of Shan State/Shan State Army-South,State of Sinai,72.727273,8245.0,ACLED,State of Sinai,unknown,unknown,violent groups
16,MILF: Moro Islamic Liberation Front,MILF: Moro Islamic Liberation Front,Oromo Liberation Front,84.210526,4888.0,GTD,Oromo Liberation Front,armed group,violent political groups,violent groups
17,Gonja Ethnic Militia (Ghana),Gonja Ethnic Militia (Ghana),Ga Ethnic Militia (Ghana),93.877551,6724.0,ACLED,Ga Ethnic Militia (Ghana),armed group,violent political groups,violent groups
25,Olo Communal Militia (Nigeria),Olo Communal Militia (Nigeria),Ologba Communal Militia (Nigeria),94.915254,7811.0,ACLED,Ologba Communal Militia (Nigeria),armed group,violent political groups,violent groups
...,...,...,...,...,...,...,...,...,...,...
3441,Ode Militia,Musa Sudi Yalahow Militia,Ode Militia,77.777778,4851.0,GTD,Musa Sudi Yalahow Militia,armed group,violent political groups,violent groups
3450,Military Forces of Venezuela (1998-) DGCIM: Directorate General of Military Counterintelligence,Forces of General Morgan,Military Forces of Venezuela (1998-) DGCIM: Directorate General of Military Counterintelligence,82.926829,4610.0,GED,Forces of General Morgan,armed group,violent political groups,violent groups
3459,CNPSC: National Coalition of the People for Congo's Sovereignty,Amani Coalition,CNPSC: National Coalition of the People for Congo's Sovereignty,75.000000,3192.0,ACLED,Amani Coalition,political party,nonviolent political groups,nonviolent groups
3469,Rohingya extremists,Right-wing extremists,Rohingya extremists,80.000000,5478.0,GTD,Right-wing extremists,armed group,violent political groups,violent groups


In [174]:
tryitout = final_actor_df_for_Gary_May_2.merge(trial_bin, left_on='missing_actors', right_on='missing_actors', how='left')[['missing_actors','Level_1',"Level_2","Level_3"]]
tryitout

,missing_actors,Level_1,Level_2,Level_3
0,Lela Chanco Communal Militia (Guatemala),NaN,NaN,NaN
1,LeJ: Lashkar-e-Jabbar,NaN,NaN,NaN
2,Police Forces of Myanmar (2016-) Border Guard Police,NaN,NaN,NaN
3,UPLC,political party,nonviolent political groups,nonviolent groups
4,Police Forces of Ethiopia (2018-) Liyu Police,security forces,violent political groups,violent groups
...,...,...,...,...
3524,Rohingya extremists,armed group,violent political groups,violent groups
3525,Rohingya extremists,armed group,violent political groups,violent groups
3526,Qeerroo Youth Militia,armed group,violent political groups,violent groups
3527,Indigenous Militia (Mexico),NaN,NaN,NaN


In [115]:
tryitout.loc[tryitout['missing_actors'] == 'LeJ: Lashkar-e-Jabbar'].index


Int64Index([], dtype='int64')

In [117]:
tryitout

,missing_actors,Level_1,Level_2,Level_3
0,Lela Chanco Communal Militia (Guatemala),armed group,NaN,NaN
1,LeJ: Lashkar-e-Jabbar,NaN,NaN,NaN
2,Police Forces of Myanmar (2016-) Border Guard Police,NaN,NaN,NaN
3,UPLC,political party,nonviolent political groups,nonviolent groups
4,Police Forces of Ethiopia (2018-) Liyu Police,security forces,violent political groups,violent groups
...,...,...,...,...
3524,Rohingya extremists,armed group,violent political groups,violent groups
3525,Rohingya extremists,armed group,violent political groups,violent groups
3526,Qeerroo Youth Militia,armed group,violent political groups,violent groups
3527,Indigenous Militia (Mexico),armed group,NaN,NaN


In [56]:
pd.concat([merged_filled_side,merged_filled_match])[pd.concat([merged_filled_side,merged_filled_match])['Level_1'].notna()]['missing_actors']

3                                                                                                  UPLC
13                                  RCSS/SSA-S: Restoration Council of Shan State/Shan State Army-South
16                                                                  MILF: Moro Islamic Liberation Front
17                                                                         Gonja Ethnic Militia (Ghana)
25                                                                       Olo Communal Militia (Nigeria)
                                                     ...                                               
3441                                                                                        Ode Militia
3450    Military Forces of Venezuela (1998-) DGCIM: Directorate General of Military Counterintelligence
3459                                    CNPSC: National Coalition of the People for Congo's Sovereignty
3469                                                            

In [60]:
no_level = []
for i in final_actor_df_for_Gary_May_2['missing_actors']:
    if i not in pd.concat([merged_filled_side,merged_filled_match])[pd.concat([merged_filled_side,merged_filled_match])['Level_1'].notna()]['missing_actors'].to_list():
        no_level.append(i)


In [62]:
len(no_level)

2389

In [55]:
temp_same_below_95.merge(actor_taxonomy, left_on='side_sort', right_on='base.categories', how='left')

,side_sort,match_sort,score_sort,usesless/meaningless,data.source,base.categories,Level_1,Level_2,Level_3
0,,,0.000000,1,NaN,NaN,NaN,NaN,NaN
1,,1st Coastal Division,0.000000,2,NaN,NaN,NaN,NaN,NaN
2,,1st Infantry Division,0.000000,3,NaN,NaN,NaN,NaN,NaN
3,,1st Regiment,0.000000,4,NaN,NaN,NaN,NaN,NaN
4,,1st Coastal Division,0.000000,2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10232,United Democratic Front (UDF),United People's Democratic Front (UPDF) - Bangladesh,92.000000,6020,GTD,United Democratic Front (UDF),armed group,violent political groups,violent groups
10233,Urhobo Gbagbako,Urhobo Revolutionary Army,57.142857,9321,GTD,Urhobo Gbagbako,armed group,violent political groups,violent groups
10234,West Nile Bank Front (WNBF),Western Somalia Liberation Front,59.649123,9327,GTD,West Nile Bank Front (WNBF),armed group,violent political groups,violent groups
10235,Zimbabwe African Nationalist Union (ZANU),Zimbabwe African People's Union,83.018868,9341,GTD,Zimbabwe African Nationalist Union (ZANU),armed group,violent political groups,violent groups


In [44]:
list1_for_side_below95 = []
for i in temp_same_below_95['match_sort']:
    if (i not in actor_taxonomy['base.categories'].to_list()) and (i not in list(set(temp_actor_final_list_for_Gary))):
        list1_for_side_below95.append(i)

list1_for_match_below95 = []
for i in temp_same_below_95['match_sort']:
    if (i not in actor_taxonomy['base.categories'].to_list()) and (i not in list(set(temp_actor_final_list_for_Gary))):
        list1_for_match_below95.append(i)



In [45]:
# Below 95 results from side and match 
final_list_for_Gary_May_2 = list(set(list1_for_side_below95 + list1_for_match_below95 + list(set(temp_actor_final_list_for_Gary))))

In [46]:
final_actor_df_for_Gary_May_2 = pd.DataFrame(final_list_for_Gary_May_2, columns = ['missing_actors'])
# final_actor_df_for_Gary_May_2.to_excel("final_actor_df_for_Gary_May_2.xlsx")


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [121]:
# As requested, I will find key words between each level of coding and try match the 
#grouped_multiple_levels = actor_taxonomy.groupby(['Level_3', 'Level_2',"Level_1"]).count()
Level_3 = actor_taxonomy['Level_3'].unique()
Level_2 = actor_taxonomy['Level_2'].unique()
Level_1 = actor_taxonomy['Level_1'].unique()

In [122]:
level1_dict = {}
for i in Level_1:
    if i not in level1_dict:
        level1_dict[i] = []
for i in range(len(actor_taxonomy)):
    if actor_taxonomy['Level_1'][i] in level1_dict:
        level1_dict[actor_taxonomy['Level_1'][i]].append(actor_taxonomy['base.categories'][i])

In [142]:
tryitout['missing_actors'].to_list().index('Indigenous Militia (Mexico)')

3527

In [145]:
pd.isna(tryitout.iloc[3527,1])

True

In [175]:

for i in tryitout['missing_actors'].to_list():
    row_indexes = tryitout['missing_actors'].to_list().index(i)
    if pd.isna(tryitout.iloc[row_indexes,1]) == True:
        if [ele for ele in ['Extermists','Militia'] if(ele in i)]:
            tryitout.loc[tryitout['missing_actors'] == i, 'Level_1'] = 'armed group'
        elif [ele for ele in ['Party','Democratic','National','Democracy','Movement','Congress'] if(ele in i)]:
            tryitout.loc[tryitout['missing_actors'] == i, 'Level_1'] = 'political party'
        elif [ele for ele in ['Forces','Military','Government','Republic','Unit'] if(ele in i)]:
            tryitout.loc[tryitout['missing_actors'] == i, 'Level_1'] = 'military'
        elif [ele for ele in ['Union','Mission'] if(ele in i)]:
            tryitout.loc[tryitout['missing_actors'] == i, 'Level_1'] = 'peacekeepers'
        elif [ele for ele in ['United','Nations','Agents','Commission'] if(ele in i)]:
            tryitout.loc[tryitout['missing_actors'] == i, 'Level_1'] = 'government'
        elif [ele for ele in ['Supporters','Civilians'] if(ele in i)]:
            tryitout.loc[tryitout['missing_actors'] == i, 'Level_1'] = 'civilians'
        elif [ele for ele in ['Islamic','Christian','Brotherhood','Christians'] if(ele in i)]:
            tryitout.loc[tryitout['missing_actors'] == i, 'Level_1'] = 'religous group'
        elif [ele for ele in ['Group','Ethnic','Alliance'] if(ele in i)]:
            tryitout.loc[tryitout['missing_actors'] == i, 'Level_1'] = 'ethnic group'
        elif [ele for ele in ['Police','Security','Guards','Private'] if(ele in i)]:
            tryitout.loc[tryitout['missing_actors'] == i, 'Level_1'] = 'security forces'
        elif [ele for ele in ['Rioters','rioters','Galamseyers','Private'] if(ele in i)]:
            tryitout.loc[tryitout['missing_actors'] == i, 'Level_1'] = 'violent civilians'
        elif [ele for ele in ['Protesters','Demonstrators','Galamseyers','Private'] if(ele in i)]:
            tryitout.loc[tryitout['missing_actors'] == i, 'Level_1'] = 'protesters'
        else:
            tryitout.loc[tryitout['missing_actors'] == i, 'Level_1'] = 'unknown'




In [176]:
def for_level_2(level_1):
    if level_1 in ["military", "security forces", "armed group", "peacekeepers"]:
        return 'violent political groups'
    elif level_1 in ['government', 'political party']:
        return 'nonviolent political groups'
    elif level_1 in ['violent civilians']:
        return 'violent civilian groups'
    elif level_1 in ['ethnic group', "religious group", "civilians", "movements", 'protesters']:
        return 'nonviolent civilian groups'
    else:
        return 'unknown'

In [177]:
for i in tryitout['missing_actors'].to_list():
    row_indexes = tryitout['missing_actors'].to_list().index(i)
    if pd.isna(tryitout.iloc[row_indexes,2]) == True:
        tryitout['Level_2'] = tryitout['Level_1'].apply(for_level_2)


In [178]:
def for_level_3(level_1):
    if level_1 in ["military", "security forces", "armed group", "peacekeepers", "violent civilians", "unknown"]:
        return 'violent groups'
    elif level_1 in ['government', 'political party', 'ethnic group', "religious group", "civilians", "movements", 'protesters']:
        return 'nonviolent groups'

In [179]:
for i in tryitout['missing_actors'].to_list():
    row_indexes = tryitout['missing_actors'].to_list().index(i)
    if pd.isna(tryitout.iloc[row_indexes,3]) == True:
        tryitout['Level_3'] = tryitout['Level_1'].apply(for_level_3)

In [161]:
"armed group" == tryitout.loc[tryitout['missing_actors'] == 'Indigenous Militia (Mexico)']['Level_1']

3527    True
Name: Level_1, dtype: bool

In [180]:
tryitout

,missing_actors,Level_1,Level_2,Level_3
0,Lela Chanco Communal Militia (Guatemala),armed group,violent political groups,violent groups
1,LeJ: Lashkar-e-Jabbar,unknown,unknown,violent groups
2,Police Forces of Myanmar (2016-) Border Guard Police,military,violent political groups,violent groups
3,UPLC,political party,nonviolent political groups,nonviolent groups
4,Police Forces of Ethiopia (2018-) Liyu Police,security forces,violent political groups,violent groups
...,...,...,...,...
3524,Rohingya extremists,armed group,violent political groups,violent groups
3525,Rohingya extremists,armed group,violent political groups,violent groups
3526,Qeerroo Youth Militia,armed group,violent political groups,violent groups
3527,Indigenous Militia (Mexico),armed group,violent political groups,violent groups


In [182]:
final_actor_df_for_Gary_May_2

,missing_actors
0,Lela Chanco Communal Militia (Guatemala)
1,LeJ: Lashkar-e-Jabbar
2,Police Forces of Myanmar (2016-) Border Guard Police
3,UPLC
4,Police Forces of Ethiopia (2018-) Liyu Police
...,...
3255,Police Forces of Uzbekistan (2016-) Prison Guards
3256,Rohingya extremists
3257,Qeerroo Youth Militia
3258,Indigenous Militia (Mexico)


In [184]:
final_actor_df_for_Gary_May_5 = final_actor_df_for_Gary_May_2.merge(tryitout, on='missing_actors',how = 'left').drop_duplicates(subset = 'missing_actors')

In [185]:
final_actor_df_for_Gary_May_5.to_excel("final_actor_df_for_Gary_May_5_binned.xlsx")

- violent political groups:
military, security forces, armed group, peacekeepers

- nonviolent political groups:
government, political party

- violent civilian groups:
violent civilians

- nonviolent civilian groups:
ethnic group, religious group, civilians, movements, protesters

In [ ]:
key_words_dict = {'armed group':['Militia'],
                'movements':['Union','National','Movement','Workers','Communal','Congress'], 
                'unknown':[], 
                'political party':['Party','Democratic','National','Democracy','Movement','Congress'],
                'military':['Forces','Military','Government','Republic','Unit'], 
                'peacekeepers':[], 
                'government':[], 
                'civilians':[], 
                'religious group':[], 
                'ethnic group':[], 
                'security forces':[], 
                'violent civilians':[], 
                'protesters':[]
}

In [123]:
for keys in level1_dict.keys():
    for_count_level1 = []
    for i in level1_dict[keys]:
        for_count_level1.append(i.split(' '))

    flatList = []
    for elem in for_count_level1:
        flatList.extend(elem)

    from collections import Counter
    Counter = Counter(flatList)
    most_occur = Counter.most_common(10)
    print(most_occur)

[('Militia', 1602), ('Ethnic', 529), ('of', 370), ('(Somalia)', 346), ('Communal', 340), ('Sub-Clan', 218), ('(Nigeria)', 195), ('for', 177), ('the', 158), ('Mayi', 151)]
[('of', 29), ('the', 13), ('Union', 12), ('for', 11), ('National', 8), ('Group', 8), ('Movement', 8), ('Workers', 6), ('Congress', 6), ('Communal', 6)]
[('of', 4), ('Abahwebi', 1), ('baKwaMashu', 1), ('AHOI:', 1), ('Amabolu', 1), ('High', 1), ('Operational', 1), ('Integrity', 1), ('Artillery', 1), ('and', 1)]
[('Party', 59), ('for', 49), ('Democratic', 44), ('National', 40), ('of', 30), ('Democracy', 21), ('Movement', 20), ('Congress', 18), ('and', 18), ('Front', 15)]
[('of', 461), ('Forces', 266), ('Military', 262), ('Government', 135), ('Republic', 40), ('Mutiny', 39), ('Sudan', 28), ('Central', 23), ('African', 23), ('Unit', 21)]
[('in', 29), ('Mission', 28), ('African', 20), ('United', 16), ('Nations', 15), ('Republic', 12), ('the', 11), ('Union', 10), ('Central', 9), ('(2007-)', 8)]
[('of', 2), ('United', 2), ('N

In [ ]:
def temp (lstoftuples):
    for x,y in lstoftuples:
        print(x,y)

In [ ]:
temp([('of', 461), ('Forces', 266), ('Military', 262), ('Government', 135), ('Republic', 40), ('Mutiny', 39), ('Sudan', 28), ('Central', 23), ('African', 23), ('Unit', 21)])

of 461
Forces 266
Military 262
Government 135
Republic 40
Mutiny 39
Sudan 28
Central 23
African 23
Unit 21


In [ ]:
level1_dict.keys()

dict_keys(['armed group', 'movements', 'unknown', 'political party', 'military', 'peacekeepers', 'government', 'civilians', 'religious group', 'ethnic group', 'security forces', 'violent civilians', 'protesters'])

In [ ]:
level1_dict.values()

13

In [ ]:
flatList = []
for elem in level1_dict['armed group']:
    flatList.extend(elem)
from collections import Counter
Counter = Counter(flatList)
most_occur = Counter.most_common(4)
print(most_occur)

[('i', 10356), ('a', 9467), (' ', 9225), ('n', 4799)]


In [ ]:
from collections import Counter
# Pass the split_it list to instance of Counter class.
Counter = Counter(flatList)

In [ ]:
most_occur = Counter.most_common(20)

In [ ]:
most_occur

[('Militia', 1602),
 ('Ethnic', 529),
 ('of', 370),
 ('(Somalia)', 346),
 ('Communal', 340),
 ('Sub-Clan', 218),
 ('(Nigeria)', 195),
 ('for', 177),
 ('the', 158),
 ('Mayi', 151),
 ('Clan', 136),
 ('Front', 122),
 ('Liberation', 120),
 ('Movement', 118),
 ('Republic', 117),
 ('(Sudan)', 108),
 ('Congo)', 107),
 ('(Democratic', 106),
 ('Forces', 96),
 ('Unidentified', 94)]

In [ ]:
armed = []
for i in actor_taxonomy['base.categories']:
    if 'Militia' in i:
        armed.append(i)


In [ ]:
armed

['"Al Shabaab" Militia',
 '"Janjaweed" Militia',
 '"Taliban" Militia',
 'Aandonga Ethnic Militia (Namibia)',
 'Aasi Clan Militia (Somalia)',
 'Abacwamba Ethnic Militia (Uganda)',
 'Abairege Ethnic Militia (Kenya)',
 'Abala Ethnic Militia (Sudan)',
 'Abanyabasi Ethnic Militia (Kenya)',
 'Abbey Ethnic Militia (Ivory Coast)',
 'Abdalla Clan Militia (Somalia)',
 'Abdalla-Agonyar Sub-Clan Militia (Somalia)',
 'Abdalla-Arab Sub-Clan Militia (Somalia)',
 'Abduwak Ethnic Militia (Kenya)',
 'Abeche Communal Militia (Chad)',
 'Abgal Clan Militia (Somalia)',
 'Abgal-Abdalla Arone Sub-Clan Militia (Somalia)',
 'Abgal-Agonyar Sub-Clan Militia (Somalia)',
 'Abgal-Daud Clan Militia (Somalia)',
 'Abgal-Hawiye Sub-Clan Militia (Somalia)',
 'Abgal-Kabale Sub-Clan Militia (Somalia)',
 'Abgal-Mohamed Muse Sub-Clan Militia (Somalia)',
 'Abgal-Reer Matan Sub-Clan Militia (Somalia)',
 'Abgal-Wabudhan-Mohamed Muse Sub-Clan Militia (Somalia)',
 'Abgal-Warsangeli Sub-Clan Militia (Somalia)',
 'Abidji Ethnic Mil

In [ ]:
# Find potential matching for sources Maybe try to find better score metric for this???

In [ ]:
# Maybe Try using MELTT tmr and see how it goes(Rstudio)